## LangChain 使用（基于DeepSeek）

### 0.安装LangChain相关包 （只需要执行一次）

In [1]:
!pip install langchain -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [ ]:
!pip install langchain-deepseek -i https://pypi.tuna.tsinghua.edu.cn/simple

### 指定Key

In [1]:
import os
os.environ["DEEPSEEK_API_BASE"] = 'https://api.deepseek.com/v1'
os.environ["DEEPSEEK_API_KEY"] = 'sk-60e9910f323c46c3aeb645951b2d45ec'

### 定义模型

In [2]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### 执行问答

In [3]:
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to Chinese."),
    ("human", "I love programming."),
]
model.invoke(messages)

AIMessage(content='我喜欢编程。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 21, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 21}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '3a13433e-a230-4b38-a1e3-129f8dd5bef2', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d09f29d3-e40f-4c41-af82-d82724836c21-0', usage_metadata={'input_tokens': 21, 'output_tokens': 3, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 1. LangChain消息不同角色

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="将以下内容翻译为英文"),
    HumanMessage(content="你好"),
]

response = model.invoke(messages)

In [7]:
print(response.content)

Hello


In [8]:
print(response)

content='Hello' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'e40d6e3f-f0bd-4131-aa35-823cb93fe695', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--c2d44f37-e148-4ef7-8a3d-d3515cd2ae2b-0' usage_metadata={'input_tokens': 11, 'output_tokens': 1, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


In [9]:
type(response)

langchain_core.messages.ai.AIMessage

### 2. LangChain中输出处理

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [11]:
parser.invoke(response)

'Hello'

### 3. LangChain 链

In [14]:
chain = model | parser

In [15]:
chain.invoke(messages)

'Hello'

### 4. LangChain中prompt模板

In [16]:
from langchain_core.prompts import ChatPromptTemplate

In [17]:
system_template = "将以下内容翻译为{language}:"

In [18]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [19]:
result = prompt_template.invoke({"language": "意大利语", "text": "你好"})

print(result)

messages=[SystemMessage(content='将以下内容翻译为意大利语:', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好', additional_kwargs={}, response_metadata={})]


In [20]:
result.to_messages()

[SystemMessage(content='将以下内容翻译为意大利语:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好', additional_kwargs={}, response_metadata={})]

In [21]:
chain = prompt_template | model | parser

In [22]:
chain.invoke({"language": "英文", "text": "你好"})

'Hello'

In [23]:
chain.invoke({"language": "法语", "text": "你好"})

'Bonjour'

In [24]:
chain.invoke({"language": "意大利语", "text": "你好"})

'Ciao'

## 5. 基于LangChain构建Chatbot

In [25]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好！我是Leo")])

AIMessage(content='你好，Leo！很高兴认识你！😊  \n我是DeepSeek，一个AI助手，由深度求索公司创造。无论你想聊天、寻求帮助，还是需要解答问题，我都会尽力为你提供支持。  \n有什么我可以帮你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 8, 'total_tokens': 60, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'a0520cd5-1d38-4919-a890-0a4a61563a8e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--2ef8ce02-d075-42d4-8762-af47f723d808-0', usage_metadata={'input_tokens': 8, 'output_tokens': 52, 'total_tokens': 60, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 但是模型不具有记忆

In [26]:
model.invoke([HumanMessage(content="我叫什么名字?")])

AIMessage(content='我无法知道您的真实姓名，因为我们的对话是匿名的。😊\n\n如果您愿意的话，可以告诉我您希望我怎么称呼您，或者直接告诉我您的名字，这样我就可以在后续对话中用您喜欢的名字来称呼您了。\n\n有什么其他我可以帮助您的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 8, 'total_tokens': 65, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '719b3116-a446-44bf-9736-9ca6b640570b', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--0396a40f-e2a3-4788-940b-0e6fb7b5596d-0', usage_metadata={'input_tokens': 8, 'output_tokens': 57, 'total_tokens': 65, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 结合上下文对话，可以输出正确结果

In [27]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="你好！我是Leo"),
        AIMessage(content="你好，Leo！很高兴见到你。有什么我可以帮助你的吗？"),
        HumanMessage(content="我叫什么名字?"),
    ]
)

AIMessage(content='你刚才提到你的名字是 **Leo**。如果这不是你希望使用的名字，或者需要我更改称呼，请随时告诉我！ 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 30, 'total_tokens': 59, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 30}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '5f68c047-da83-4781-9f8c-86d92c692690', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--e1dc1acf-98b5-4188-a5fc-b238006ada74-0', usage_metadata={'input_tokens': 30, 'output_tokens': 29, 'total_tokens': 59, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 消息历史

In [28]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [29]:
config = {"configurable": {"session_id": "abc1"}}

In [30]:
response = with_message_history.invoke(
    [HumanMessage(content="你好！我是Leo")],
    config=config,
)

response.content

'你好，Leo！很高兴认识你！😊  \n我是DeepSeek，一个AI助手，由深度求索公司创造。无论你想聊天、寻求帮助，还是需要解答问题，我都会尽力为你提供支持。  \n有什么我可以帮你的吗？'

In [31]:
response = with_message_history.invoke(
    [HumanMessage(content="我叫什么名字?")],
    config=config,
)

response.content

'你刚才告诉我，你的名字是 **Leo**！😊  \n如果这不是你真实的名字，或者你想让我称呼你别的名字，请随时告诉我～  \n有什么需要我帮忙的吗，Leo？'

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="我有一条宠物狗，它的名字是旺财")],
    config=config,
)

response.content

'哈哈，旺财真是个好名字！听起来特别有福气又亲切～🐶  \n它是什么品种的狗狗呀？平时有什么有趣的小故事可以分享吗？😄'

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="我的宠物叫什么?")],
    config=config,
)

response.content

'根据我们之前的对话，你提到你的宠物狗名字是 **旺财**～🐕  \n需要我帮你记录更多关于旺财的信息吗？或者想聊聊养宠物的趣事？😊'

### 流式输出

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("告诉我一个关于 {topic} 的故事")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "猫"}):
    print(chunk, end="", flush=True)

# 最后的猫语者

在遥远的北方，有一座被遗忘的村庄，村里住着一位名叫艾尔莎的老妇人。她是村里最后一位“猫语者”——传说中能与猫对话的人。村里的年轻人都搬去了城市，只剩下老人和几十只猫。

艾尔莎记得每只猫的名字：有独眼的“船长”，三条腿的“小跳”，还有总是忧郁地望着远方的“灰云”。每天黄昏，她坐在门廊的摇椅上，猫咪们围着她，听她讲述古老的故事。

“很久以前，”她会用沙哑的声音说，“猫不是宠物，而是信使。它们能在梦境中穿梭，传递人们不敢说出口的话语。”

一个寒冷的冬夜，村里来了一位陌生的年轻人。他叫利奥，是艾尔莎从未谋面的孙子。利奥的父亲多年前离开村庄，再也没有回来。

“奶奶，”利奥说，“爸爸临终前让我把这个交给您。”他递上一封泛黄的信。

艾尔莎颤抖着打开信，泪水模糊了字迹。信上写着：“母亲，请原谅我。我找到了能治愈猫语者家族遗传病的方法，但研究需要时间。告诉利奥，他的使命是保护最后的猫。”

原来，艾尔莎家族的女性都活不过七十岁，这是一种罕见的遗传病。她的女儿——利奥的母亲——也因此在年轻时去世。

那天夜里，艾尔莎发起了高烧。利奥焦急地想找医生，但暴风雪封住了出村的路。猫咪们围在艾尔莎床边，“船长”轻轻舔着她的手。

“灰云”突然跳到利奥面前，琥珀色的眼睛紧紧盯着他。利奥感到一阵眩晕，然后发现自己站在一片白茫茫的梦境中。

“利奥，”一个声音说，“我是灰云。艾尔莎的时间不多了，但你可以救她。”

在梦境中，利奥得知了一个秘密：猫语者的能力不仅在于与猫对话，还能通过猫的梦境传递生命力。但这样做需要一只猫自愿牺牲自己的九条命之一。

“用我的第一条命，”灰云平静地说，“艾尔莎救过我的命，现在是我回报的时候。”

利奥在灰云的引导下，学会了古老的仪式。当黎明第一缕阳光照进小屋时，艾尔莎的烧退了，而灰云显得疲惫但安详。

康复后的艾尔莎将一切告诉了利奥：“我们的家族使命不是与猫对话，而是守护它们与人类世界之间脆弱的桥梁。现在，轮到你了。”

春天来临时，艾尔莎在睡梦中安详离世，猫咪们围在她身边。利奥决定留在村庄，继续祖母的使命。他惊讶地发现自己也能听懂猫的语言——不是通过遗传，而是通过爱与责任。

如今，当旅人经过那座北方村庄，总会看到一位年轻人和一群猫坐在门廊上。如果你仔细听，风声中似乎有轻柔的对话，那是人类与猫之间，持续千年的低语。

而灰云依然喜欢望着远方